您将向状态添加额外的字段，以定义复杂的行为，而无需依赖消息列表

1. 向状态添加键

In [1]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt

# 定义状态类，继承自 TypedDict
class State(TypedDict):
    # messages: 消息列表字段
    # Annotated[list, add_messages] 表示这是一个列表，使用 add_messages 函数来合并新旧消息
    # add_messages 会智能地将新消息追加到现有消息列表中，而不是替换
    messages: Annotated[list, add_messages]
    
    # name: 存储名称的字符串字段（普通字段，直接替换更新）
    name: str
    
    # birthday: 存储生日的字符串字段（普通字段，直接替换更新）
    birthday: str

在工具内部更新状态

现在，在 human_assistance 工具内部填充状态键。这允许人工在信息存储到状态之前进行审查。使用 Command 从工具内部发出状态更新指令。

In [2]:


@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

In [3]:
# 导入必要的模块
import os  # 用于环境变量操作
from langchain_openai import ChatOpenAI  # 导入 OpenAI 聊天模型
import dotenv

# 加载 .env 文件（如果存在）
dotenv.load_dotenv() 
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL") 

# # 可选的基础 URL，如果存在则设置
# base_url = os.getenv("OPENAI_BASE_URL")
# os.environ['OPENAI_BASE_URL'] = base_url

# 初始化语言模型（只初始化一次）
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [4]:

tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

3. 提示聊天机器人

提示聊天机器人查找 LangGraph 库的“生日”，并指示聊天机器人在获得所需信息后联系 human_assistance 工具。通过在工具的参数中设置 name 和 birthday，您可以强制聊天机器人为这些字段生成建议。

In [5]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

In [6]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [14]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_JHGkSX7y1yCWqb0X4oDE92GY)
 Call ID: call_JHGkSX7y1yCWqb0X4oDE92GY
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search

{'error': ValueError("Error 432: This request exceeds this API key's set usage limit. You can increase its limit on the Tavily dashboard.")}
================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_zwffIj8bDAx3aRNCQOU5Pwam)
 Call ID: call_zwffIj8bDAx3aRNCQOU5Pwam
  Args:
    name: LangGraph
    birthday: 2023-09-01


In [11]:
snapshot = graph.get_state(config)
snapshot.next
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{}

4. 添加人工协助¶

In [15]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_zwffIj8bDAx3aRNCQOU5Pwam)
 Call ID: call_zwffIj8bDAx3aRNCQOU5Pwam
  Args:
    name: LangGraph
    birthday: 2023-09-01
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================

LangGraph was released on January 17, 2024.


In [ ]:
snapshot = graph.get_state(config)
snapshot.values.items()
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

dict_items([('messages', [HumanMessage(content='Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.', additional_kwargs={}, response_metadata={}, id='a10230db-bc5f-401d-a162-3fc79f8505e8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p6oGoHZI9uPry2ypw13RTvq7', 'function': {'arguments': '{"query":"LangGraph release date"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 1318, 'total_tokens': 1338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CToVAZeJJJpHcSfssHCp1v2uK5XLl', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a

5. 手动更新状态¶

LangGraph 提供了对应用程序状态的高度控制。例如，在任何时候（包括中断时），您都可以使用 graph.update_state 手动覆盖一个键。

In [19]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0b0128-a314-6854-800d-b0a829aac7de'}}

查看新值

如果您调用 graph.get_state，您可以看到新值已反映出来。

In [20]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}